# Segmenting and Clustering Neighbourhoods in Toronto

## Excercise 1

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import urllib

In [2]:
# fetching data from wikipedia page and storing into canada_data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
canada_data = requests.get(url).text
 
tabla = canada_data[canada_data.find("<table"):canada_data.find("</table>")+8]
df = pd.read_html(tabla, header = 0)[0]

In [3]:
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [4]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df = df[df.Borough != "Not assigned"]

In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.Neighbourhood[df.Neighbourhood == "Not assigned"] = df.Borough[df.Neighbourhood == "Not assigned"]

In [6]:
#Rows will be combined by Postcode to compose the name of all neighbourhoods
def neighbourhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = df.groupby(['Postal Code', 'Borough'])
newDf = grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')

In [7]:
#the number of rows of dataframe
newDf.shape

(103, 3)

In [8]:
#Cleaned Dataframe
# Changing column name from Postcode to PostalCode
newDf = newDf.rename(columns = {'Postcode':'PostalCode'})
newDf.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
#Export dataframe to new CSV file for excercise 2
newDf.to_csv('Toronto_Part_I_dataframe.cvs', index=False)

## Excercise 2

In [10]:
!pip install geocoder

In [11]:
import numpy as np
import pandas as pd
import geocoder

In [12]:
#Use the Geocoder to find coordinates
#Take M5G as an example
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords


get_latlng('M5G')

[43.65609000000006, -79.38492999999994]

In [13]:
#Use CSV file to add coordinates
#Use a given link of csv file that has the geographical coordinates of each postal code
url='http://cocl.us/Geospatial_data'
geo_data = pd.read_csv(url).set_index("Postal Code")
geo_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [14]:
#Using neighbourhoods in toronto dataframe that created in Excercise (which stored in csv file)
toronto_data = pd.read_csv('Toronto_Part_I_dataframe.cvs').set_index("Postal Code")
toronto_data.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [15]:
#Joining both dataframes together
joinData = toronto_data.join(geo_data)
joinData.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
#Export dataframe to new CSV file for excercise 3
joinData.to_csv('Toronto_Part_II_dataframe.cvs',index=False)

In [17]:

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

In [18]:
#Load data from from CSV file which created in Excercise 2
neighborhoods = pd.read_csv('Toronto_Part_II_dataframe.cvs')
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
#Use geopy library to get the latitude and longitude values of Toronto city
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
#simplify the above map and segment and cluster only the neighborhoods in North York. Slice the original dataframe and create a new dataframe of the North York data.
northyork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Hillcrest Village,43.803762,-79.363452
1,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,North York,Bayview Village,43.786947,-79.385975
3,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [22]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [23]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [24]:
#start utilizing the Foursquare API to explore the neighborhoods and segment them
CLIENT_ID = 'XXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXXXXXXXXXXXXXXX
CLIENT_SECRET:XXXXXXXXXXX


In [25]:
#Get the neighborhood's name
northyork_data.loc[0, 'Neighbourhood']

'Hillcrest Village'

In [26]:
#Get the neighborhood's latitude and longitude values.
neighbourhood_latitude = northyork_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = northyork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = northyork_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


**Get the top 100 venues that are in Hillcrest Village within a radius of 500 meters**

In [27]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5f18438d1db3e514e9667c14'},
 'response': {}}

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']